# Design Calculations

Starting this to capture some calculations I need to make to design the robot

In [16]:
# How much do we need to lift?
# What's a reasonable expectation for the amount of weight we can lift?

# assuming the weight of the arm itself is evenly distributed
arm_weight_lbs = 6
arm_weight_kg = arm_weight_lbs * 0.453592

# let's look at the limit of the arm being fully extended
fully_extended_arm_in = 24
fully_extended_arm_m = fully_extended_arm_in * 0.0254

def torque_with_arm_weight(payload_mass, length=fully_extended_arm_m, arm_mass=arm_weight_kg):
    # we need to account for the weight of the arm itself
    # so we need to treat the arm as a lever with mass and with a fulcrum at the shoulder
    # and the payload as a weight at the end of the lever
    gravity = 9.81  # Earth's gravity in m/s^2

    # Torque due to the arm's weight
    # Arm's weight acts at the midpoint of the arm
    arm_weight = arm_mass * gravity
    arm_torque = arm_weight * (length / 2)

    # Torque due to the payload's weight
    # Payload's weight acts at the end of the arm
    payload_weight = payload_mass * gravity
    payload_torque = payload_weight * length

    # Total torque is the sum of both torques
    total_torque = arm_torque + payload_torque

    return total_torque

payload_mass = 5  # kg
max_torque = torque_with_arm_weight(payload_mass)

print(f"Max torque with {payload_mass} kg payload: {max_torque:.2f} Nm")

Max torque with 5 kg payload: 38.04 Nm


In [29]:
input_gear = 10
output_gear = 220
motor_holding_torque = 1.9
rated_current = 2.8

gear_ratio = output_gear / input_gear
output_holding_torque_2AM = motor_holding_torque * gear_ratio

def cost_of_electricity_per_hour(amps, voltage=12, cost_per_kwh=0.15):
    return (amps * voltage) / 1000 * cost_per_kwh

def coeph(amps, voltage=12, cost_per_kwh=0.15):
    return cost_of_electricity_per_hour(amps, voltage, cost_per_kwh)

def coep_month(amps, voltage=12, cost_per_kwh=0.15):
    return coeph(amps, voltage, cost_per_kwh) * 8 * 30

print(f"Gear ratio: {gear_ratio}")
print(f"Motor holding torque: {motor_holding_torque} Nm")
print(f"Output holding torque: {output_holding_torque_2AM} Nm @ ${coeph(rated_current):.3f} per hour")

Gear ratio: 22.0
Motor holding torque: 1.9 Nm
Output holding torque: 41.8 Nm @ $0.005 per hour


In [32]:
input_rpm = 100
output_rpm = input_rpm / gear_ratio

def rpm_to_degrees_per_second(rpm):
    return rpm * 360 / 60

print(f"Output RPM: {output_rpm:.2f} / {rpm_to_degrees_per_second(output_rpm):.2f} deg/s")

Output RPM: 4.55 / 27.27 deg/s


In [59]:
# Can we calculate alternate gear ratios? Ie., instead of 2 gears (10:220), what if we had 3 gears (10:100:110)?
def calculate_gear_ratio(gears):
    # print(f"Gear ratio: {gears}")
    if len(gears) < 2:
        raise ValueError("At least two gears are required for a gear ratio calculation.")

    overall_ratio = 1.0

    for i in range(len(gears) - 1):
        driving_gear = gears[i]
        driven_gear = gears[i + 1]

        if driving_gear == 0:
            raise ValueError("A gear cannot have zero teeth.")

        ratio = driven_gear / driving_gear
        overall_ratio *= ratio
        # print(f"Gear {i + 1} ratio: {ratio} - Overall ratio: {overall_ratio}")

    return overall_ratio

print(f"Gear ratio: {calculate_gear_ratio([10, 220])}")
print(f"Gear ratio: {calculate_gear_ratio([10, 32])}")

Gear ratio: 22.0
Gear ratio: 0.9411764705882353


In [57]:
print(3.2 * 6.875)

22.0
